# Anthropogenic heat from vehicular traffic

This notebook is used to process population data, and ultimately produce the anthropogenic heat flux (AHF) from vehicular traffic for the Greater Los Angeles (LA) region.

In [1]:
# load packages 
%matplotlib inline
import os
import glob
from pathlib import Path
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
#import requests
import geopandas as gpd
import pyproj
import shapely.geometry
import contextily as ctx
from matplotlib.ticker import FormatStrFormatter
import geofeather
import dask_geopandas as dgpd
import pyogrio
import folium
import numpy as np

In [8]:
# import link inventory (i.e. locations and atributes)
inv_path = '/Users/josephko/USC/Research/Anthropogenic Heat/adms/congestion_inv_all_distinct_links.csv'
link_inv_df = pd.read_csv(inv_path)
link_inv_gdf = gpd.GeoDataFrame(link_inv_df, 
                                geometry=gpd.points_from_xy(link_inv_df.start_location_lng, link_inv_df.start_location_lat))
link_inv_gdf.set_crs(crs='EPSG:4326', inplace=True)
link_inv_gdf

,link_id,agency,city,link_type,on_street,from_street,to_street,start_location_lng,start_location_lat,direction,postmile,num_lanes,geometry
0,760650,Caltrans-D7,NaN,HIGHWAY,I-110,AMADOR,NaN,-118.232562,34.075049,SOUTH,25.00,5,POINT (-118.23256 34.07505)
1,810997,Caltrans-D8,NaN,HIGHWAY,10,M 1.0 W/O 4th STREET,NaN,-117.635171,34.083132,EAST,4.16,4,POINT (-117.63517 34.08313)
2,4078-E,LADOT,NaN,ARTERIAL,AVIATION BL,CENTURY BL,NaN,-118.378845,33.945362,EAST,0.00,5,POINT (-118.37884 33.94536)
3,775068,Caltrans-D7,NaN,HIGHWAY,I-5,PIONEER,NaN,-118.082820,33.921239,SOUTH,5.25,3,POINT (-118.08282 33.92124)
4,120764,LADOT,NaN,ARTERIAL,JEFFERSON BL,PLAYA VISTA DR,NaN,-118.428847,33.973710,SOUTH,0.00,4,POINT (-118.42885 33.97371)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88253,816382,Caltrans-D8,NaN,HIGHWAY,10,CALIFORNIA ST.,NaN,-117.229799,34.066939,EAST,28.17,4,POINT (-117.22980 34.06694)
88254,40147,LADOT,NaN,ARTERIAL,LA TIJERA BL,SEPULVEDA BL,NaN,-118.397119,33.957073,NORTH,0.00,4,POINT (-118.39712 33.95707)
88255,801577,Caltrans-D8,NaN,HIGHWAY,215,BOX SPRINGS,NaN,-117.301111,33.949468,NORTH,30.60,4,POINT (-117.30111 33.94947)
88256,60180,LADOT,NaN,ARTERIAL,HAYVENHURST AV,VENTURA BL,NaN,-118.492955,34.157672,WEST,0.00,4,POINT (-118.49295 34.15767)


In [9]:
# export as shapefile 
link_inv_gdf.to_file('/Users/josephko/USC/Research/Anthropogenic Heat/adms/link_inv_gdf.shp')

/Users/josephko/opt/anaconda3/envs/geo_env/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
/var/folders/4m/_rb2pz1d7jx9ywsn916lhr_c0000gn/T/ipykernel_8945/392666212.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  link_inv_gdf.to_file('/Users/josephko/USC/Research/Anthropogenic Heat/adms/link_inv_gdf.shp')


In [ ]:
# import 